<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

assosiate rule learning

In [1]:
pip install mlxtend


In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

transactions = [
    ['milk', 'bread', 'butter'],
    ['beer', 'bread'],
    ['milk', 'bread', 'butter'],
    ['beer', 'bread', 'butter'],
    ['milk', 'bread'],
    ['beer', 'bread']
]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

print("Frequent Itemsets:")
print(frequent_itemsets)

print("\nAssociation Rules:")
print(rules)


Frequent Itemsets:
   support         itemsets
0      0.5           (beer)
1      1.0          (bread)
2      0.5         (butter)
3      0.5           (milk)
4      0.5    (bread, beer)
5      0.5  (bread, butter)
6      0.5    (bread, milk)

Association Rules:
  antecedents consequents  antecedent support  consequent support  support  \
0     (bread)      (beer)                 1.0                 0.5      0.5   
1      (beer)     (bread)                 0.5                 1.0      0.5   
2     (bread)    (butter)                 1.0                 0.5      0.5   
3    (butter)     (bread)                 0.5                 1.0      0.5   
4     (bread)      (milk)                 1.0                 0.5      0.5   
5      (milk)     (bread)                 0.5                 1.0      0.5   

   confidence  lift  leverage  conviction  zhangs_metric  
0         0.5   1.0       0.0         1.0            0.0  
1         1.0   1.0       0.0         inf            0.0  
2         0.5

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Neural Collaborative filtering

In [4]:
pip install torch torchvision


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [5]:
 import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Dummy dataset
class NCFDataset(Dataset):
    def __init__(self, user_item_matrix):
        self.user_item_matrix = user_item_matrix
        self.users, self.items = np.nonzero(user_item_matrix)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        user = self.users[idx]
        item = self.items[idx]
        rating = self.user_item_matrix[user, item]
        return user, item, rating

# NCF Model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)

        self.fc_layers = nn.Sequential(
            nn.Linear(latent_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        x = torch.cat([user_embedding, item_embedding], dim=-1)
        x = self.fc_layers(x)
        return x

# Sample data
num_users = 100
num_items = 50
latent_dim = 8

user_item_matrix = np.random.randint(2, size=(num_users, num_items))

dataset = NCFDataset(user_item_matrix)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = NCF(num_users, num_items, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for user, item, rating in dataloader:
        user = user.long()
        item = item.long()
        rating = rating.float()

        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

print("Training complete!")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/10, Loss: 0.17278393543077797
Epoch 2/10, Loss: 0.013519892439576266
Epoch 3/10, Loss: 0.007267190612074506
Epoch 4/10, Loss: 0.004201037106339973
Epoch 5/10, Loss: 0.002694900149827281
Epoch 6/10, Loss: 0.001794071225567332
Epoch 7/10, Loss: 0.0012660618352739116
Epoch 8/10, Loss: 0.0009675088480020626
Epoch 9/10, Loss: 0.0007192134032692006
Epoch 10/10, Loss: 0.0005837882267712038
Training complete!


Hybrid recommendation systems

In [6]:
pip install numpy pandas scikit-learn surprise


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357251 sha256=c9e93deaefaeebca27e0cbd0b8638910dc58bcd001548bf3046011647d986a50
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [7]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load data
ratings_data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'item_id': [101, 102, 103, 101, 102, 104, 103, 104, 105],
    'rating': [5, 4, 3, 4, 5, 4, 3, 5, 4]
}

item_data = {
    'item_id': [101, 102, 103, 104, 105],
    'title': ['Item A', 'Item B', 'Item C', 'Item D', 'Item E'],
    'description': ['A great item', 'An awesome item', 'A fantastic item', 'A superb item', 'A magnificent item']
}

ratings_df = pd.DataFrame(ratings_data)
items_df = pd.DataFrame(item_data)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(items_df['description'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def hybrid_recommend(user_id, item_id, top_n=5):
    collab_pred = algo.predict(user_id, item_id).est

    item_idx = items_df[items_df['item_id'] == item_id].index[0]
    sim_scores = list(enumerate(cosine_sim[item_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    content_based_recommendations = [i[0] for i in sim_scores[1:top_n+1]]

    content_pred = items_df.iloc[content_based_recommendations].to_dict('records')

    return {
        'Collaborative Filtering Prediction': collab_pred,
        'Content-Based Recommendations': content_pred
    }

user_id = 1
item_id = 105

recommendations = hybrid_recommend(user_id, item_id)
print(recommendations)


{'Collaborative Filtering Prediction': 3.7221028039785558, 'Content-Based Recommendations': [{'item_id': 101, 'title': 'Item A', 'description': 'A great item'}, {'item_id': 102, 'title': 'Item B', 'description': 'An awesome item'}, {'item_id': 103, 'title': 'Item C', 'description': 'A fantastic item'}, {'item_id': 104, 'title': 'Item D', 'description': 'A superb item'}]}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
